In [1]:
import torch
import pandas as pd
import os
import librosa
import contextlib
from mutagen.mp3 import MP3
from datasets import load_dataset, Value, Features, Audio, Dataset, concatenate_datasets
from sklearn.preprocessing import minmax_scale
import numpy as np
import evaluate
import texthero as hero
from transformers import set_seed
torch.cuda.empty_cache()
set_seed(42)
# Details: https://huggingface.co/docs/diffusers/optimization/fp16#enable-cudnn-autotuner
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
from transformers import pipeline
os.cpu_count()

D:\Downloads\Swahili-translation-English-STT--main\ASR\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


20

In [2]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
# print(torch.cuda.current_device())
# print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('NOT connected to a GPU!. Will use CPU!')
else:
  print('Connected to a GPU!')

True
1
NVIDIA GeForce RTX 3070 Ti
Connected to a GPU!


In [3]:
# !pip install huggingface_hub
# !pip install ipywidgets
# !pip install git+https://github.com/huggingface/datasets
# !pip install git+https://github.com/huggingface/transformers
# !pip install librosa
# !pip install pandas openpyxl
# !pip install numpy
# !pip install audiomentations
# !pip install mutagen

Make Dataset

In [4]:
# abs_path = ".\\LARGE DATA\\larger sample\\"
# df = pd.ExcelFile(".\LARGE DATA\larger sample\Compiled data - submission 1.xlsx")
# df1 = pd.read_excel(df, 'QF14')
# df2 = pd.read_excel(df, 'QF18')
# df3 = pd.read_excel(df, 'QG17')
# df4 = pd.read_excel(df, 'QH11')
# df5 = pd.read_excel(df, 'QH12')

# df = pd.concat([df1, df2, df3, df4, df5], ignore_index = True)
# df

In [5]:
# for i in range(len(df)):
#     if "QF14" in df['File name'][i]:
#         df['File name'][i] = abs_path+"QF14\\"+df['File name'][i]
#     elif "QF18" in df['File name'][i]:
#         df['File name'][i] = abs_path+"QF18\\"+df['File name'][i]
#     elif "QG17" in df['File name'][i]:
#         df['File name'][i] = abs_path+"QG17\\"+df['File name'][i]
#     elif "QH11" in df['File name'][i]:
#         df['File name'][i] = abs_path+"QH11\\"+df['File name'][i]
#     elif "QH12" in df['File name'][i]:
#         df['File name'][i] = abs_path+"QH12\\"+df['File name'][i]
# df['File name'][225]

In [6]:
# # df.to_excel('Swahili_QF.xlsx', index=False)
# df.rename(columns={'File name': 'file_name'}, inplace=True)

In [7]:
df = pd.read_excel("./real_data.xlsx")

In [8]:
df.shape

(1184, 2)

In [9]:
print(df['file_name'][642])
df['transcription'][642]

./LARGE DATA/larger sample/QG17/GK_swa_u0709_QG17.mp3


"Moraa should have just agreed to be paid through the method of The phone so that her debt can be deducted and get exhausted and then Should opt out from the loans she has because she ends up losing clients When it's something simple. You can just pay your loan and then just opt ������out. You get yourself off that [?]Go on with your life as usual instead of [p]"

In [10]:
print(df['file_name'][555])
df['transcription'][555]

./LARGE DATA/larger sample/QG17/GK_swa_u0479_QG17.mp3


'In my opinion Moraa finds it difficult to pay the debt she took Through her phone and maybe she did not have enough so she feels if she is paid through the phone The money will be taken to pay off the debt and she will not be able to stock up [null]'

In [11]:
print(df['file_name'][331])
df['transcription'][331]

./LARGE DATA/larger sample/QF18/GK_swa_u0525_QF18.mp3


'It is good if my credit increases'

In [12]:
# sf = pd.read_excel(".\real_data.xlsx")
# # sf = pd.read_excel(".\\Swahili_QF_new.xlsx")

In [13]:
type(df['transcription'][22])

str

In [14]:
print(df['file_name'][655])
df['transcription'][655]

./LARGE DATA/larger sample/QG17/GK_swa_u0731_QG17.mp3


"I think she should come up with Pochi la biashara or till number whereby The money I won't be charged. [?] I can use but still, I don't have any debt. The money isn't deducted. They don't rhyme."

In [15]:
print(df['file_name'][27])
df['transcription'][27]

./LARGE DATA/larger sample/QF14/GK_swa_u0257_QF14.mp3


'The first is to protect yourself against fraud, the second'

In [16]:
audio_dir = "/Swahili_STT"
csv_dir = "Swahili_QF.xlsx"
audio_t = ".mp3"
base_dir = "./caches_large"

In [17]:
import re
def st_pre(df):
    df['transcription'] = hero.remove_diacritics(df['transcription'])
    df['transcription'] = hero.remove_whitespace(df['transcription'])
    for i in range(len(df)):
        df['transcription'][i] = re.sub("[\(\[].*?[\)\]]", "", df['transcription'][i])
        df['transcription'][i] = re.sub(r"(?<=\w)([A-Z])()", r" \1", df['transcription'][i])
    df['transcription'] = hero.remove_whitespace(df['transcription'])
    return df

In [18]:
for i in range(len(df)):
    if df['transcription'][i] == 'nan':
        df['transcription'][i] = None
    elif df['transcription'][i] == '[foreign]':
        df['transcription'][i] = None
    elif df['transcription'][i] == '[null]':
        df['transcription'][i] = None
    elif df['transcription'][i] == '[?]':
        df['transcription'][i] =  None
    elif '�' in df['transcription'][i]:
        df['transcription'][i] =  None



In [19]:
# df = pd.read_excel('complete_compiled.xlsx')
print(df['file_name'][642])
print(df['transcription'][642])

./LARGE DATA/larger sample/QG17/GK_swa_u0709_QG17.mp3
None


In [20]:
j = 0
for i in range(len(df)):
    if df['transcription'][i] == '':
        df.drop(i, inplace = True)
        j+=1
j

0

In [21]:
df

,file_name,transcription
0,./LARGE DATA/larger sample/QF14/GK_swa_u0120_Q...,[foreign-luo] I can buy it because I also what...
1,./LARGE DATA/larger sample/QF14/GK_swa_u0123_Q...,[?] I would not tell them to [?]
2,./LARGE DATA/larger sample/QF14/GK_swa_u0126_Q...,I would like to buy it because [p] it would he...
3,./LARGE DATA/larger sample/QF14/GK_swa_u0134_Q...,It is the responsibility of the internet servi...
4,./LARGE DATA/larger sample/QF14/GK_swa_u0136_Q...,If it is a tool that can help the I will buy.
...,...,...
1179,./Swahili_STT/GK_swa_u0709_QA45.mp3,Safaricom is for Mpesa deposit then Airtel is ...
1180,./Swahili_STT/GK_swa_u0722_QA45.mp3,Safaricom sim card is for receiving money or d...
1181,./Swahili_STT/GK_swa_u0750_QA45.mp3,[p] [um] I use one sim card for transactions a...
1182,./Swahili_STT/GK_swa_u0766_QA44.mp3,"Although I have two Safaricoms, they are all o..."


In [22]:
df.dropna(inplace = True)

In [23]:
df

,file_name,transcription
0,./LARGE DATA/larger sample/QF14/GK_swa_u0120_Q...,[foreign-luo] I can buy it because I also what...
1,./LARGE DATA/larger sample/QF14/GK_swa_u0123_Q...,[?] I would not tell them to [?]
2,./LARGE DATA/larger sample/QF14/GK_swa_u0126_Q...,I would like to buy it because [p] it would he...
3,./LARGE DATA/larger sample/QF14/GK_swa_u0134_Q...,It is the responsibility of the internet servi...
4,./LARGE DATA/larger sample/QF14/GK_swa_u0136_Q...,If it is a tool that can help the I will buy.
...,...,...
1179,./Swahili_STT/GK_swa_u0709_QA45.mp3,Safaricom is for Mpesa deposit then Airtel is ...
1180,./Swahili_STT/GK_swa_u0722_QA45.mp3,Safaricom sim card is for receiving money or d...
1181,./Swahili_STT/GK_swa_u0750_QA45.mp3,[p] [um] I use one sim card for transactions a...
1182,./Swahili_STT/GK_swa_u0766_QA44.mp3,"Although I have two Safaricoms, they are all o..."


In [24]:
# j = 0
# for i in range(len(df)):
#     if df['transcription'][i] == '':
#         j+=1
# j

In [25]:
df.dropna(inplace = True)

In [26]:
df = df.reset_index()
df.drop(columns = ['index'], inplace = True)

In [27]:
# df.drop(columns = ['index'], inplace = True)
len(df['transcription'][1])

32

In [28]:
df

,file_name,transcription
0,./LARGE DATA/larger sample/QF14/GK_swa_u0120_Q...,[foreign-luo] I can buy it because I also what...
1,./LARGE DATA/larger sample/QF14/GK_swa_u0123_Q...,[?] I would not tell them to [?]
2,./LARGE DATA/larger sample/QF14/GK_swa_u0126_Q...,I would like to buy it because [p] it would he...
3,./LARGE DATA/larger sample/QF14/GK_swa_u0134_Q...,It is the responsibility of the internet servi...
4,./LARGE DATA/larger sample/QF14/GK_swa_u0136_Q...,If it is a tool that can help the I will buy.
...,...,...
1178,./Swahili_STT/GK_swa_u0709_QA45.mp3,Safaricom is for Mpesa deposit then Airtel is ...
1179,./Swahili_STT/GK_swa_u0722_QA45.mp3,Safaricom sim card is for receiving money or d...
1180,./Swahili_STT/GK_swa_u0750_QA45.mp3,[p] [um] I use one sim card for transactions a...
1181,./Swahili_STT/GK_swa_u0766_QA44.mp3,"Although I have two Safaricoms, they are all o..."


In [29]:
for i in range(len(df)):
    if df['transcription'][i].count(" ") < 5:
        df.drop(i, inplace = True)

In [30]:
df

,file_name,transcription
0,./LARGE DATA/larger sample/QF14/GK_swa_u0120_Q...,[foreign-luo] I can buy it because I also what...
1,./LARGE DATA/larger sample/QF14/GK_swa_u0123_Q...,[?] I would not tell them to [?]
2,./LARGE DATA/larger sample/QF14/GK_swa_u0126_Q...,I would like to buy it because [p] it would he...
3,./LARGE DATA/larger sample/QF14/GK_swa_u0134_Q...,It is the responsibility of the internet servi...
4,./LARGE DATA/larger sample/QF14/GK_swa_u0136_Q...,If it is a tool that can help the I will buy.
...,...,...
1178,./Swahili_STT/GK_swa_u0709_QA45.mp3,Safaricom is for Mpesa deposit then Airtel is ...
1179,./Swahili_STT/GK_swa_u0722_QA45.mp3,Safaricom sim card is for receiving money or d...
1180,./Swahili_STT/GK_swa_u0750_QA45.mp3,[p] [um] I use one sim card for transactions a...
1181,./Swahili_STT/GK_swa_u0766_QA44.mp3,"Although I have two Safaricoms, they are all o..."


In [31]:
df = df.reset_index()

In [32]:
df['transcription'][642]

"In my opinion, I see that she should repay the loan because now it looks like she will lose because no customer She won't allow payment via the phone because of her debt. It would have been better if she repays the debt or she just agrees to repay the debt[?] so that she can look for more customers"

In [33]:
df.drop(columns = ['index'], inplace = True)

In [34]:
sf = pd.read_excel("large_subset_small_instance.xlsx")
sf

,Unnamed: 0,file_name,transcription
0,0,./Smaller instances/QA10/GK_swa_u0120_QA10.mp3,1984
1,1,./Smaller instances/QA10/GK_swa_u0123_QA10.mp3,1986
2,2,./Smaller instances/QA10/GK_swa_u0126_QA10.mp3,1986
3,3,./Smaller instances/QA10/GK_swa_u0130_QA10.mp3,1992
4,4,./Smaller instances/QA10/GK_swa_u0134_QA10.mp3,1995
...,...,...,...
6986,6986,./Smaller instances/QE90/GK_swa_u0807_QE90.mp3,"Once a month, it depends, if its Certificate o..."
6987,6987,./Smaller instances/QE90/GK_swa_u0809_QE90.mp3,"I hardly use it, because E C I T I Z E N I reg..."
6988,6988,./Smaller instances/QE93/GK_swa_u0264_QE93.mp3,My doubt is that there are hackers in the worl...
6989,6989,./Smaller instances/QE93/GK_swa_u0406_QE93.mp3,Information


In [35]:
sf.drop(columns = ['Unnamed: 0'], inplace = True)
sf

,file_name,transcription
0,./Smaller instances/QA10/GK_swa_u0120_QA10.mp3,1984
1,./Smaller instances/QA10/GK_swa_u0123_QA10.mp3,1986
2,./Smaller instances/QA10/GK_swa_u0126_QA10.mp3,1986
3,./Smaller instances/QA10/GK_swa_u0130_QA10.mp3,1992
4,./Smaller instances/QA10/GK_swa_u0134_QA10.mp3,1995
...,...,...
6986,./Smaller instances/QE90/GK_swa_u0807_QE90.mp3,"Once a month, it depends, if its Certificate o..."
6987,./Smaller instances/QE90/GK_swa_u0809_QE90.mp3,"I hardly use it, because E C I T I Z E N I reg..."
6988,./Smaller instances/QE93/GK_swa_u0264_QE93.mp3,My doubt is that there are hackers in the worl...
6989,./Smaller instances/QE93/GK_swa_u0406_QE93.mp3,Information


In [36]:
for i in range(len(sf)):
    if sf['transcription'][i] == '.':
        sf.drop(i, inplace = True)
        print("n")
    elif sf['transcription'][i] == '. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .':
        sf.drop(i, inplace = True)
        print('s')

n
n
n
n
n
s
n
n
n
n


In [37]:
sf.reset_index(inplace = True)
sf.drop(columns = ['index'], inplace = True)

In [38]:
df = pd.concat([df, sf], ignore_index = True)
df.reset_index(inplace = True)
df.drop(columns = ['index'], inplace = True)

In [39]:
df

,file_name,transcription
0,./LARGE DATA/larger sample/QF14/GK_swa_u0120_Q...,[foreign-luo] I can buy it because I also what...
1,./LARGE DATA/larger sample/QF14/GK_swa_u0123_Q...,[?] I would not tell them to [?]
2,./LARGE DATA/larger sample/QF14/GK_swa_u0126_Q...,I would like to buy it because [p] it would he...
3,./LARGE DATA/larger sample/QF14/GK_swa_u0134_Q...,It is the responsibility of the internet servi...
4,./LARGE DATA/larger sample/QF14/GK_swa_u0136_Q...,If it is a tool that can help the I will buy.
...,...,...
8148,./Smaller instances/QE90/GK_swa_u0807_QE90.mp3,"Once a month, it depends, if its Certificate o..."
8149,./Smaller instances/QE90/GK_swa_u0809_QE90.mp3,"I hardly use it, because E C I T I Z E N I reg..."
8150,./Smaller instances/QE93/GK_swa_u0264_QE93.mp3,My doubt is that there are hackers in the worl...
8151,./Smaller instances/QE93/GK_swa_u0406_QE93.mp3,Information


In [40]:
# df.drop(columns = ['level_0'], inplace = True)

In [41]:
df = st_pre(df)

In [42]:
df

,file_name,transcription
0,./LARGE DATA/larger sample/QF14/GK_swa_u0120_Q...,I can buy it because I also what phone theft t...
1,./LARGE DATA/larger sample/QF14/GK_swa_u0123_Q...,I would not tell them to
2,./LARGE DATA/larger sample/QF14/GK_swa_u0126_Q...,I would like to buy it because it would help m...
3,./LARGE DATA/larger sample/QF14/GK_swa_u0134_Q...,It is the responsibility of the internet servi...
4,./LARGE DATA/larger sample/QF14/GK_swa_u0136_Q...,If it is a tool that can help the I will buy.
...,...,...
8148,./Smaller instances/QE90/GK_swa_u0807_QE90.mp3,"Once a month, it depends, if its Certificate o..."
8149,./Smaller instances/QE90/GK_swa_u0809_QE90.mp3,"I hardly use it, because E C I T I Z E N I reg..."
8150,./Smaller instances/QE93/GK_swa_u0264_QE93.mp3,My doubt is that there are hackers in the worl...
8151,./Smaller instances/QE93/GK_swa_u0406_QE93.mp3,Information


In [43]:
j = 0
for i in range(len(df)):
    if df['transcription'][i] == '':
        df.drop(i, inplace = True)
        j+=1
j

1

In [44]:
df = df.reset_index()

In [45]:
df = df.drop(['index'], axis = 1)

In [46]:
# df = df.drop('level_0', axis = 1)

In [47]:
df

,file_name,transcription
0,./LARGE DATA/larger sample/QF14/GK_swa_u0120_Q...,I can buy it because I also what phone theft t...
1,./LARGE DATA/larger sample/QF14/GK_swa_u0123_Q...,I would not tell them to
2,./LARGE DATA/larger sample/QF14/GK_swa_u0126_Q...,I would like to buy it because it would help m...
3,./LARGE DATA/larger sample/QF14/GK_swa_u0134_Q...,It is the responsibility of the internet servi...
4,./LARGE DATA/larger sample/QF14/GK_swa_u0136_Q...,If it is a tool that can help the I will buy.
...,...,...
8147,./Smaller instances/QE90/GK_swa_u0807_QE90.mp3,"Once a month, it depends, if its Certificate o..."
8148,./Smaller instances/QE90/GK_swa_u0809_QE90.mp3,"I hardly use it, because E C I T I Z E N I reg..."
8149,./Smaller instances/QE93/GK_swa_u0264_QE93.mp3,My doubt is that there are hackers in the worl...
8150,./Smaller instances/QE93/GK_swa_u0406_QE93.mp3,Information


In [48]:
print(df['file_name'][200])
df['transcription'][200]

./LARGE DATA/larger sample/QF14/GK_swa_u0754_QF14.mp3


"I really have no idea about that, but I think, I don't know"

In [49]:
print(df['file_name'][800])
df['transcription'][800]

./LARGE DATA/larger sample/QH11/GK_swa_u0602_QH11.mp3


'Benefits are that it makes you it makes you exposed to many people In the whole country and the world so a lot of people could know Your products and decide to purchase them and also begin to believe that your products are good.'

In [50]:
print(df['file_name'][801])
df['transcription'][801]

./LARGE DATA/larger sample/QH11/GK_swa_u0603_QH11.mp3


'I feel like they are there in the business to reach people and by the end of the day there is a purchase And people that sent the money were customers you had not planned for It has importance after all there are challenges But at the end of the day you will get clients'

In [51]:
print(df['file_name'][501])
df['transcription'][501]

./LARGE DATA/larger sample/QG17/GK_swa_u0284_QG17.mp3


'I have an opinion that Moraa needs to repay her loan so that her limit increases She can get her limit has increased and be able to buy products to stock to her shop'

In [52]:
print(df['file_name'][666])
df['transcription'][666]

./LARGE DATA/larger sample/QG17/GK_swa_u0774_QG17.mp3


"I only find it as a reason, My opinion is that Moraa hasn't Has not been exposed. If she could be well conversant with business, she could have a 'Lipa na Mpesa'. If She had the 'Lipa na Mpesa', they would use the till number. That money Obviously can not be reversed because it goes to Safaricom. So, I think Moraa didn't use her mind when opening her business"

In [53]:
print(df['file_name'][234])
df['transcription'][234]

./LARGE DATA/larger sample/QF14/GK_swa_u0804_QF14.mp3


"I haven't known about such a service"

In [54]:
print(df['file_name'][456])
df['transcription'][456]

./LARGE DATA/larger sample/QF18/GK_swa_u0788_QF18.mp3


"If I check, I can't say There are two different ways to check them. One, it will be okay to increase that loan because You'll find that they even give someone permission to borrow more. if I really have the ability to pay. If you look at it from the other side, it is not right to increase the limit Because that will now make a person have the desire to take more loans, and maybe he does not have the ability to pay"

In [55]:
df['transcription'][456][-1]

'y'

In [56]:
df

,file_name,transcription
0,./LARGE DATA/larger sample/QF14/GK_swa_u0120_Q...,I can buy it because I also what phone theft t...
1,./LARGE DATA/larger sample/QF14/GK_swa_u0123_Q...,I would not tell them to
2,./LARGE DATA/larger sample/QF14/GK_swa_u0126_Q...,I would like to buy it because it would help m...
3,./LARGE DATA/larger sample/QF14/GK_swa_u0134_Q...,It is the responsibility of the internet servi...
4,./LARGE DATA/larger sample/QF14/GK_swa_u0136_Q...,If it is a tool that can help the I will buy.
...,...,...
8147,./Smaller instances/QE90/GK_swa_u0807_QE90.mp3,"Once a month, it depends, if its Certificate o..."
8148,./Smaller instances/QE90/GK_swa_u0809_QE90.mp3,"I hardly use it, because E C I T I Z E N I reg..."
8149,./Smaller instances/QE93/GK_swa_u0264_QE93.mp3,My doubt is that there are hackers in the worl...
8150,./Smaller instances/QE93/GK_swa_u0406_QE93.mp3,Information


In [57]:
df['transcription'].dropna(inplace=True)

In [58]:
df

,file_name,transcription
0,./LARGE DATA/larger sample/QF14/GK_swa_u0120_Q...,I can buy it because I also what phone theft t...
1,./LARGE DATA/larger sample/QF14/GK_swa_u0123_Q...,I would not tell them to
2,./LARGE DATA/larger sample/QF14/GK_swa_u0126_Q...,I would like to buy it because it would help m...
3,./LARGE DATA/larger sample/QF14/GK_swa_u0134_Q...,It is the responsibility of the internet servi...
4,./LARGE DATA/larger sample/QF14/GK_swa_u0136_Q...,If it is a tool that can help the I will buy.
...,...,...
8147,./Smaller instances/QE90/GK_swa_u0807_QE90.mp3,"Once a month, it depends, if its Certificate o..."
8148,./Smaller instances/QE90/GK_swa_u0809_QE90.mp3,"I hardly use it, because E C I T I Z E N I reg..."
8149,./Smaller instances/QE93/GK_swa_u0264_QE93.mp3,My doubt is that there are hackers in the worl...
8150,./Smaller instances/QE93/GK_swa_u0406_QE93.mp3,Information


In [59]:
df.dropna(inplace = True)
# df = df.reset_index()
# df = df.drop(['index'], axis = 1)

In [60]:
print(df['file_name'][2340])
df['transcription'][2340]

./Smaller instances/QA15/GK_swa_u0767_QA15.mp3


'Yes, I do go to work in other places as you cannot be in one place all the time'

In [61]:
print(df['file_name'][2899])
df['transcription'][2899]

./Smaller instances/QA19/GK_swa_u0731_QA19.mp3


'There 8 children, 5 go to school and 3 don’t go because of school fees'

In [62]:
df['transcription'][0]

'I can buy it because I also what phone theft to end'

In [63]:
for i in range(len(df)):
    if df['transcription'][i][-1] == ' ':
        df['transcription'][i][-1] = '.' 
    elif df['transcription'][i][-1] != '.':
        df['transcription'][i] = df['transcription'][i]+'.'
    elif df['transcription'][i][0] == '.':
        df['transcription'][i] = df['transcription'][i][1:]
        

In [64]:
print(df['file_name'][6500])
df['transcription'][6500]

./Smaller instances/QD21/GK_swa_u0791_QD21.mp3


'Nothing.'

In [65]:
print(df['file_name'][4500])
df['transcription'][4500]

./Smaller instances/QC12/GK_swa_u0225_QC12.mp3


'Several times in a day.'

In [66]:
print(df['file_name'][3999])
df['transcription'][3999]

./Smaller instances/QA53/GK_swa_u0680_QA53.mp3


'I always go to the side.'

In [67]:
print(df['file_name'][2500])
df['transcription'][2500]

./Smaller instances/QA18/GK_swa_u0581_QA18.mp3


'In a day is 200 shillings.'

In [68]:
print(df['file_name'][801])
df['transcription'][801]

./LARGE DATA/larger sample/QH11/GK_swa_u0603_QH11.mp3


'I feel like they are there in the business to reach people and by the end of the day there is a purchase And people that sent the money were customers you had not planned for It has importance after all there are challenges But at the end of the day you will get clients.'

In [69]:
print(df['file_name'][1170])
df['transcription'][1170]

./Swahili_STT/GK_swa_u0766_QA45.mp3


'My two Safaricom sim cards are well. I use them to call people. I use them like in M-pesa and the Telkom sim card that I have now, I use it for bundles because its bundles Are a little bit cheap and not like Airtel. I bought the Airtel sim card because I have clients that have Airtel So, I bought Airtel. I call Airtel customers with Airtel and Safaricom customers with Safaricom.'

In [70]:
df = st_pre(df)

In [71]:
j = 0
for i in range(len(df)):
    if df['transcription'][i] == '':
        df.drop(i, inplace = True)
        j+=1
j

0

In [72]:
df.reset_index(inplace=True)
df.drop(columns = ['index'], inplace = True)

In [73]:
# df.drop_duplicates(subset=['transcription'], keep='first', inplace = True)

In [74]:
df.columns

Index(['file_name', 'transcription'], dtype='object')

In [75]:
df

,file_name,transcription
0,./LARGE DATA/larger sample/QF14/GK_swa_u0120_Q...,I can buy it because I also what phone theft t...
1,./LARGE DATA/larger sample/QF14/GK_swa_u0123_Q...,I would not tell them to.
2,./LARGE DATA/larger sample/QF14/GK_swa_u0126_Q...,I would like to buy it because it would help m...
3,./LARGE DATA/larger sample/QF14/GK_swa_u0134_Q...,It is the responsibility of the internet servi...
4,./LARGE DATA/larger sample/QF14/GK_swa_u0136_Q...,If it is a tool that can help the I will buy.
...,...,...
8147,./Smaller instances/QE90/GK_swa_u0807_QE90.mp3,"Once a month, it depends, if its Certificate o..."
8148,./Smaller instances/QE90/GK_swa_u0809_QE90.mp3,"I hardly use it, because E C I T I Z E N I reg..."
8149,./Smaller instances/QE93/GK_swa_u0264_QE93.mp3,My doubt is that there are hackers in the worl...
8150,./Smaller instances/QE93/GK_swa_u0406_QE93.mp3,Information.


In [76]:
df.reset_index(inplace=True)
df.drop(columns = ['index'], inplace = True)

In [77]:
j = 0
for i in range(len(df)):
    if df['transcription'][i] == "":
        df.drop(i, inplace = True)
        j+=1
j

0

In [78]:
np.where(pd.isnull(df))

(array([], dtype=int64), array([], dtype=int64))

In [79]:
df

,file_name,transcription
0,./LARGE DATA/larger sample/QF14/GK_swa_u0120_Q...,I can buy it because I also what phone theft t...
1,./LARGE DATA/larger sample/QF14/GK_swa_u0123_Q...,I would not tell them to.
2,./LARGE DATA/larger sample/QF14/GK_swa_u0126_Q...,I would like to buy it because it would help m...
3,./LARGE DATA/larger sample/QF14/GK_swa_u0134_Q...,It is the responsibility of the internet servi...
4,./LARGE DATA/larger sample/QF14/GK_swa_u0136_Q...,If it is a tool that can help the I will buy.
...,...,...
8147,./Smaller instances/QE90/GK_swa_u0807_QE90.mp3,"Once a month, it depends, if its Certificate o..."
8148,./Smaller instances/QE90/GK_swa_u0809_QE90.mp3,"I hardly use it, because E C I T I Z E N I reg..."
8149,./Smaller instances/QE93/GK_swa_u0264_QE93.mp3,My doubt is that there are hackers in the worl...
8150,./Smaller instances/QE93/GK_swa_u0406_QE93.mp3,Information.


In [80]:
for i in range(len(df)): 
    # print(i)
    if df['transcription'][i] == '. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .':
        print(i)
    elif df['transcription'][i] == '.':
        print("something")


something
something


In [81]:
df.to_excel("Send_deco.xlsx")

In [82]:
df['transcription'][7989]

'. . . .'

In [83]:
df.drop([7989], inplace = True)

In [84]:
df.reset_index(inplace=True)
df.drop(columns = ['index'], inplace = True)
df

,file_name,transcription
0,./LARGE DATA/larger sample/QF14/GK_swa_u0120_Q...,I can buy it because I also what phone theft t...
1,./LARGE DATA/larger sample/QF14/GK_swa_u0123_Q...,I would not tell them to.
2,./LARGE DATA/larger sample/QF14/GK_swa_u0126_Q...,I would like to buy it because it would help m...
3,./LARGE DATA/larger sample/QF14/GK_swa_u0134_Q...,It is the responsibility of the internet servi...
4,./LARGE DATA/larger sample/QF14/GK_swa_u0136_Q...,If it is a tool that can help the I will buy.
...,...,...
8146,./Smaller instances/QE90/GK_swa_u0807_QE90.mp3,"Once a month, it depends, if its Certificate o..."
8147,./Smaller instances/QE90/GK_swa_u0809_QE90.mp3,"I hardly use it, because E C I T I Z E N I reg..."
8148,./Smaller instances/QE93/GK_swa_u0264_QE93.mp3,My doubt is that there are hackers in the worl...
8149,./Smaller instances/QE93/GK_swa_u0406_QE93.mp3,Information.


In [85]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.15, shuffle = True)

In [86]:
test

,file_name,transcription
6576,./Smaller instances/QD38/GK_swa_u0604_QD38.mp3,"It's not a worry, but maybe it's just the cert..."
483,./LARGE DATA/larger sample/QG17/GK_swa_u0207_Q...,She was afraid to receive money on phone becau...
7605,./Smaller instances/QE46/GK_swa_u0312_QE46.mp3,A few times in a week.
3936,./Smaller instances/QA49/GK_swa_u0486_QA49.mp3,There is none.
7070,./Smaller instances/QD82/GK_swa_u0303_QD82.mp3,The bad habit of people withdrawing money with...
...,...,...
247,./LARGE DATA/larger sample/QF18/GK_swa_u0177_Q...,I wouldn't say it is good but you get loan age...
5108,./Smaller instances/QC32/GK_swa_u0484_QC32.mp3,They are very annoying.
6142,./Smaller instances/QD11/GK_swa_u0284_QD11.mp3,Once a month.
865,./LARGE DATA/larger sample/QH11/GK_swa_u0728_Q...,If you look at this life we are living in noth...


In [87]:
len(test)

1223

In [88]:
test = test.drop_duplicates(subset=['transcription'], keep='first',)

In [89]:
test.columns

Index(['file_name', 'transcription'], dtype='object')

In [90]:
# test.reset_index(inplace = True)
# test.drop(columns = ["index"], inplace = True)

In [91]:
len(test)

872

In [92]:
len(test)

872

Custom Dataset using xlsx file

In [93]:
# os.environ['HF_DATASETS_CACHE'] = os.path.join(base_dir, 'data_rash')
# os.environ['HF_DATASETS_OFFLINE'] = '0'


# features = Features(
#     {   
#         # "file_name": Value('string'),
#         "file_name": Audio(sampling_rate=16000),
#         "transcription": Value("string")
#     }
# )

# swahili = Dataset.from_pandas(df, features = features, split = ['train', 'test'], 
# )
# swahili

In [92]:
# swahili[49]["transcription"]

In [93]:
# swahili[49]["file_name"]

In [94]:
# print("Path: ", swahili[49]["file_name"]['path'])
# print("Sampling rate: ", swahili[49]["file_name"]['sampling_rate'])

Dataset Understanding

In [95]:
def getWavDuration(fname):
    audio = MP3(fname)
    duration =  audio.info.length

    return duration

def duration_infor(dataset):
    list_1 = []
    list_2 = []
    text_1 = []
    for i in range(dataset.num_rows):
        duration = getWavDuration(dataset[i]['file_name']['path'])
        list_1.append(duration)
        text_1.append(len(dataset[i]['transcription']))
        if duration > 30:
            list_2.append(duration)
    print("Total Files: ", len(list_1))
    print("Minimum length of audio in dataset: ", min(list_1))
    print("Total audio time: ",  sum(list_1))
    print("Average length of audio in dataset: ", sum(list_1)/len(list_1))
    print("Maximum lenth of audio in dataset: ", max(list_1))
    print("Audio files with more than 30 sec: ", len(list_2))
    print("Max size of string: ", max(text_1))
    print("Min size of string: ", min(text_1))
#     print("Unique characters: ", )
        
duration_infor(swahili)

NameError: name 'swahili' is not defined

In [93]:
import IPython.display as ipd
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display

In [ ]:
which_file = 29
filename = swahili[which_file]["file_name"]['path']
y, sr = librosa.load(filename, sr = 16000)
print(y)
print(sr)
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
print('Estimated tempo: {:.2f} beats per minute'.format(tempo))
beat_times = librosa.frames_to_time(beat_frames, sr=sr)
print(beat_times)

In [ ]:
ipd.Audio(y, rate=sr)

In [ ]:
plt.figure(figsize=(15, 4), facecolor=(.9, .9, .9))
librosa.display.waveshow(y, sr=sr, color='pink')

Spectral Centroid

In [ ]:
# all_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)
# spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
# spectral_centroids.shape
# frames = range(len(spectral_centroids))
# t = librosa.frames_to_time(frames)
# def normalize(x, axis=0):
#     return minmax_scale(x, axis=axis)
# #Plotting the Spectral Centroid along the waveform
# librosa.display.waveshow(y, sr=sr, alpha=0.4)
# plt.plot(t, normalize(spectral_centroids), color='r')
# #Plotting the Spectral Centroid along the waveform

RMS ENERGY and LOG POWER SPECTOGRAM

In [92]:
S, phase = librosa.magphase(librosa.stft(y))
rms = librosa.feature.rms(S=S)
import matplotlib.pyplot as plt
fig, ax = plt.subplots(nrows=2, figsize=(20, 10), sharex=True)
times = librosa.times_like(rms)
ax[0].semilogy(times, rms[0], label='RMS Energy')
ax[0].set(xticks=[])
ax[0].legend()
ax[0].label_outer()
librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max), y_axis='log', x_axis='time', ax=ax[1])
ax[1].set(title='log Power spectrogram')

NameError: name 'y' is not defined

In [93]:
import re
def remove_accented_chars(text):
#     ```
#     (NFKD) will apply the compatibility decomposition, i.e. 
#     replace all compatibility characters with their equivalents. 
#     ```
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [94]:
# Regex for matching zero witdh joiner variations.
STANDARDIZE_ZW = re.compile(r'(?<=\u09b0)[\u200c\u200d]+(?=\u09cd\u09af)')

# Regex for removing standardized zero width joiner, except in edge cases.
DELETE_ZW = re.compile(r'(?<!\u09b0)[\u200c\u200d](?!\u09cd\u09af)')

## Regex matching punctuations to remove.
# PUNC = re.compile(r'([\?\.।;:,!"\'])')
## Keeps fullstop(.), dari(|), comma(,), exclamaition(!) and question mark(?) and removes all other punctuations (semicolon (;), colon (:), double quote (") and single quote (')).
PUNC = re.compile(r'([;:"\'])')

def removeOptionalZW(text):
    """
    Removes all optional occurrences of ZWNJ or ZWJ from Bangla text.
    """
    text = STANDARDIZE_ZW.sub('\u200D', text)
    text = DELETE_ZW.sub('', text)
    return text

def removePunc(text):
    """
    Remove for punctuations from text.
    """
    text = PUNC.sub(r"", text)
    return text

Dataset Split

In [94]:
# swahili = swahili.shuffle(seed = 420)

In [95]:
# swahili_split = swahili.train_test_split(test_size=0.15)
# swahili_split

In [95]:
features = Features(
    {   
        # "file_name": Value('string'),
        "file_name": Audio(sampling_rate=16000),
        "transcription": Value("string")
    }
)

In [96]:
ds_dict = {'train' : Dataset.from_pandas(train, features = features, preserve_index = False),
           'test' : Dataset.from_pandas(test, features = features, preserve_index = False)}

In [97]:
from datasets import Dataset, DatasetDict

swahili_split = DatasetDict(ds_dict)

In [98]:
swahili_split

DatasetDict({
    train: Dataset({
        features: ['file_name', 'transcription'],
        num_rows: 6928
    })
    test: Dataset({
        features: ['file_name', 'transcription'],
        num_rows: 872
    })
})

In [99]:
swahili_split['train'][0]['file_name']

{'path': './Smaller instances/QD11/GK_swa_u0595_QD11.mp3',
 'array': array([ 7.27595761e-12, -1.63709046e-11, -4.18367563e-11, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]),
 'sampling_rate': 16000}

In [100]:
swahili_split['train'][0]['transcription']

'A few times in a week.'

Augmentations

In [101]:
from audiomentations import (
    AddBackgroundNoise, #Mixes in another sound to add background noise
    AddGaussianNoise, #Adds gaussian noise to the audio samples
    Compose,
    Gain, #Multiplies the audio by a random gain factor
    OneOf,
    PitchShift, #Shifts the pitch up or down without changing the tempo
    PolarityInversion, #Flips the audio samples upside down, reversing their polarity
    Limiter, #Applies dynamic range compression limiting the audio signal
    Mp3Compression, #Compresses the audio to lower the quality
    PitchShift, # Shifts the pitch up or down without changing the tempo
    RoomSimulator,# Simulates the effect of a room on an audio source
    SpecFrequencyMask, # Applies a frequency mask to the spectrogram
    TimeMask, #Makes a random part of the audio silent
    TimeStretch, #Changes the speed without changing the pitch
    Trim, #Trims leading and trailing silence from the audio
)

# define augmentation
augmentation = Compose(
    [
        TimeStretch(min_rate=0.9, max_rate=1.1, p=0.3, leave_length_unchanged=True),
        Gain(min_gain_in_db=-6, max_gain_in_db=6, p=0.1),
        PitchShift(min_semitones=-4, max_semitones=4, p=0.1),
        AddGaussianNoise(min_amplitude=0.005, 
                         max_amplitude=0.015, 
                         p=0.1),
        Limiter(min_threshold_db=-16.0,
                max_threshold_db=-6.0,
                threshold_mode="relative_to_signal_peak",
                p=0.1,),
        # RoomSimulator(min_size_x = 2, max_size_x = 10,
        #              min_size_y = 2, min_size_y = 15,
        #              min_size_y = 7, min_size_y = 12,
        #              min_absorption_value = 0.10,
        #              max_absorption_value = 0.50, p =0.2),
        Trim(top_db=20.0,p=0.1),
        Mp3Compression(),
    ]
)

In [102]:
swahili_split

DatasetDict({
    train: Dataset({
        features: ['file_name', 'transcription'],
        num_rows: 6928
    })
    test: Dataset({
        features: ['file_name', 'transcription'],
        num_rows: 872
    })
})

In [103]:
def augment_dataset(batch):
    # load and (possibly) resample audio data to 16kHz
    sample = batch['file_name']

    # apply augmentation
    augmented_waveform = augmentation(sample["array"], sample_rate=sample["sampling_rate"])
    batch["file_name"]["array"] = augmented_waveform
    return batch


In [104]:
# torch.set_num_threads(1)
augmented_raw_training_dataset = swahili_split["train"].map(
    augment_dataset, 
    num_proc= 1, 
    desc="augment train dataset",
    load_from_cache_file=True,
    # batched = True,
    # writer_batch_size=3_000,
    cache_file_name=os.path.join(base_dir, "./Data_aug_dont_drop.arrow")
)

In [105]:
# augmented_raw_training_dataset

In [106]:
print("\n COMBINING Augmented Dataset with Normal Dataset..... \n")
# combine
swahili_split["train"] = concatenate_datasets([swahili_split["train"], augmented_raw_training_dataset])



 COMBINING Augmented Dataset with Normal Dataset..... 



In [107]:
print("\n\n AFTER AUGMENTATION, FINAL train and validation sets are: ")
print("\n FINAL DATASET: \n")
swahili_split["train"] = swahili_split["train"].shuffle(seed=42)
print(swahili_split)



 AFTER AUGMENTATION, FINAL train and validation sets are: 

 FINAL DATASET: 

DatasetDict({
    train: Dataset({
        features: ['file_name', 'transcription'],
        num_rows: 13856
    })
    test: Dataset({
        features: ['file_name', 'transcription'],
        num_rows: 872
    })
})


In [108]:
def print_model_size(mdl):
    torch.save(mdl.state_dict(), "tmp.pt")
    print("%.2f MB" %(os.path.getsize("tmp.pt")/1e6))
    os.remove('tmp.pt')

In [109]:
swahili_split['train'][0]

{'file_name': {'path': None,
  'array': array([0., 0., 0., ..., 0., 0., 0.]),
  'sampling_rate': 16000},
 'transcription': 'Several times a day.'}

In [110]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Swahili", task="translate")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
forced_decoder_ids = processor.get_decoder_prompt_ids(language="Swahili", task="translate")
# max_length = model.max_length()
#Model Architecture and parameters
print("Parameters: ", model.num_parameters())
# size = print_model_size(model)
# print(size[0])
model

Parameters:  241734912


WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (f

In [111]:
# from transformers import WhisperProcessor, WhisperForConditionalGeneration

# processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Arabic", task="translate")
# model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small", load_in_8bit = True, device_map="auto")
# forced_decoder_ids = processor.get_decoder_prompt_ids(language="Arabic", task="translate")
# # max_length = model.max_length()
# #Model Architecture and parameters
# print("Parameters: ", model.num_parameters())
# # size = print_model_size(model)
# # print(size[0])
# model

In [111]:
forced_decoder_ids

[(1, 50318), (2, 50358), (3, 50363)]

In [112]:
# forced_decoder_ids = None
# from functools import partial
# model.generate = partial(model.generate, language="Arabic", task="translate")

In [113]:
processor.tokenizer.language

'swahili'

In [114]:
# model.config.apply_spec_augment = True

In [115]:
max_label_length = model.config.max_length
generation_max_length = max_label_length

In [116]:
generation_max_length

448

In [117]:
model.config.apply_spec_augment = True
# model.config.mask_time_prob = 0.1
# model.config.mask_feature_prob = 0.1
model.config.max_length = generation_max_length
model.config.dropout = 0.1
model.config.forced_decoder_ids = forced_decoder_ids
model.config.suppress_tokens = []
# if gradient_checkpointing:
#     model.config.use_cache = False
# if freeze_feature_encoder:
#     model.freeze_feature_encoder()
# model.config.activation_dropout=0.055,
    # attention_dropout=0.094,
model.generation_config.max_length = generation_max_length

In [119]:
# inputs = processor.feature_extractor(
#         swahili_split['train']['file_name'][0]["array"], 
#         sampling_rate=swahili_split['train']['file_name'][0]["sampling_rate"], 
#         return_attention_mask=apply_spec_augment,
#         )
# inputs

In [118]:
apply_spec_augment = True

def prepare_dataset(batch):
    # load and (possibly) resample audio data to 16kHz
    audio = batch["file_name"]

    # compute log-Mel input features from input audio array 
    inputs = processor.feature_extractor(
        audio["array"], 
        sampling_rate=audio["sampling_rate"], 
        return_attention_mask=apply_spec_augment,
        )
    batch["input_features"] = inputs.input_features[0]

    # compute input length
    batch["input_length"] = len(audio["array"]) / audio["sampling_rate"]
    
    # if spec augmentation applied, get attention_mask to guide the mask along time axis
    if apply_spec_augment:
        batch["attention_mask"] = inputs.get("attention_mask")[0]
    
    
    # optional pre-processing steps
    transcription = batch["transcription"]
#     if do_lower_case:
#         transcription = transcription.lower()
#     if do_remove_punctuation:
#         transcription = normalizer(transcription).strip()
#     if do_bangla_unicode_normalization:
#         _words = [bangla_normalizer(word)['normalized'] for word in transcription.split()]
#         transcription = " ".join([word for word in _words if word is not None])
#         transcription = transcription.replace("\u2047", "-")
#         transcription = transcription.replace(u"\u098c", u"\u09ef")
#         transcription = unicodedata.normalize("NFC", transcription)
#         transcription = removeOptionalZW(transcription)
    
    # encode target text to label ids
    batch["labels"] = processor.tokenizer(transcription).input_ids
    
     # compute labels length **with** special tokens! -> total label length
    batch["labels_length"] = len(batch["labels"])
    
    return batch

In [119]:
swahili_split

DatasetDict({
    train: Dataset({
        features: ['file_name', 'transcription'],
        num_rows: 13856
    })
    test: Dataset({
        features: ['file_name', 'transcription'],
        num_rows: 872
    })
})

In [120]:
# from datasets import set_caching_enabled
# set_caching_enabled(False)
processed_swahili =  swahili_split.map(prepare_dataset, 
                            num_proc= 1, # if num_proc>1, then mapping might get stuck. use num_proc=1 in that case.
                            load_from_cache_file=True,
                            cache_file_names={
                                "train" : os.path.join(base_dir)+'/complete_data_dondrop.arrow',
                                "test" : os.path.join(base_dir)+'/complete_data_test_dondrop.arrow', }
                            )

In [121]:
processed_swahili

DatasetDict({
    train: Dataset({
        features: ['file_name', 'transcription', 'input_features', 'input_length', 'attention_mask', 'labels', 'labels_length'],
        num_rows: 13856
    })
    test: Dataset({
        features: ['file_name', 'transcription', 'input_features', 'input_length', 'attention_mask', 'labels', 'labels_length'],
        num_rows: 872
    })
})

In [122]:
processed_swahili['train']['input_length'][45]

5.688

In [123]:
processed_swahili['train']['transcription'][45]

'3500 shillings.'

In [124]:
processed_swahili

DatasetDict({
    train: Dataset({
        features: ['file_name', 'transcription', 'input_features', 'input_length', 'attention_mask', 'labels', 'labels_length'],
        num_rows: 13856
    })
    test: Dataset({
        features: ['file_name', 'transcription', 'input_features', 'input_length', 'attention_mask', 'labels', 'labels_length'],
        num_rows: 872
    })
})

In [125]:
def filter_labels(labels_length):
    """Filter label sequences longer than max length (448)"""
    return labels_length < max_label_length

swahili_f = processed_swahili.filter(filter_labels, input_columns=["labels_length"])

Filter: 100%|██████████████████████████████████████████████████████████████| 872/872 [00:00<00:00, 67071.94 examples/s]


In [126]:
swahili_f

DatasetDict({
    train: Dataset({
        features: ['file_name', 'transcription', 'input_features', 'input_length', 'attention_mask', 'labels', 'labels_length'],
        num_rows: 13854
    })
    test: Dataset({
        features: ['file_name', 'transcription', 'input_features', 'input_length', 'attention_mask', 'labels', 'labels_length'],
        num_rows: 872
    })
})

In [127]:
max_input_length = 30.0
min_input_length = 1.0

def is_audio_in_length_range(length):
    return length < max_input_length and length > min_input_length
swahili_final = swahili_f.filter(is_audio_in_length_range, input_columns=["input_length"]
                               )

Filter: 100%|██████████████████████████████████████████████████████████████| 872/872 [00:00<00:00, 34544.82 examples/s]


In [128]:
swahili_final

DatasetDict({
    train: Dataset({
        features: ['file_name', 'transcription', 'input_features', 'input_length', 'attention_mask', 'labels', 'labels_length'],
        num_rows: 12960
    })
    test: Dataset({
        features: ['file_name', 'transcription', 'input_features', 'input_length', 'attention_mask', 'labels', 'labels_length'],
        num_rows: 807
    })
})

In [131]:
# MAX_DURATION_IN_SECONDS = 30.0
# max_input_length = MAX_DURATION_IN_SECONDS * 16000

# def filter_inputs(input_length):
#     """Filter inputs with zero input length or longer than 30s"""
#     return 0 < input_length < max_input_length

# swahili_f = processed_swahili.filter(filter_inputs, input_columns=["input_length"])

In [133]:
# def filter_labels(labels_length):
#     """Filter label sequences longer than max length (448)"""
#     return labels_length < max_label_length

# swahili_final = swahili_f.filter(filter_labels, input_columns=["labels_length"])

In [129]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    forward_attention_mask: bool

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        if self.forward_attention_mask:
            batch["attention_mask"] = torch.LongTensor([feature["attention_mask"] for feature in features])
        
        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [130]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor, forward_attention_mask=apply_spec_augment)

In [132]:
from peft import prepare_model_for_int8_training
model.enable_input_require_grads()
model = prepare_model_for_int8_training(model)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
binary_path: D:\Downloads\Swahili-translation-English-STT--main\ASR\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll
CUDA SETUP: Loading binary D:\Downloads\Swahili-translation-English-STT--main\ASR\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll...


D:\Downloads\Swahili-translation-English-STT--main\ASR\lib\site-packages\peft\utils\other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [133]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(r=64, lora_alpha=128, target_modules=["q_proj", "v_proj"], lora_dropout=0.1, bias="none")

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 7,077,888 || all params: 248,812,800 || trainable%: 2.8446639401188363


In [134]:
from transformers import Seq2SeqTrainingArguments

peft_args = Seq2SeqTrainingArguments(
    output_dir="./whisper_small/peft_large",
    overwrite_output_dir=True,
    max_steps= 12000,
    # num_train_epochs = 15,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=20,
    gradient_checkpointing=True,
    evaluation_strategy= 'steps',
    eval_steps= 400,
    save_strategy='steps',
    save_steps= 400,
    save_total_limit= 5,
    learning_rate=1e-4,
    lr_scheduler_type = "cosine", # "constant", "constant_with_warmup", "cosine", "cosine_with_restarts", "linear"(default), "polynomial", "inverse_sqrt"
    warmup_steps= 600,
    logging_steps= 5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    # metric_for_best_model= "bleu",
    # greater_is_better=True,
    # bf16= True,
#     tf32= True,
    # fp16 = True,
    optim="adamw_hf",
    torch_compile = False,
    generation_max_length=max_label_length,
    report_to=['tensorboard'],
    # generation_num_beams = 4,
#     fsdp ,
#     fsdp_config,
#   deepspeed
    #eval_delay=
    data_seed = 420,
    logging_dir = "./whisper_small/peft_large/logs",
    seed = 420,
    remove_unused_columns = False,
    label_names=["labels"],  # same reason as above

    # predict_with_generate= True,
#     push_to_hub=push_to_hub,
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [135]:
from transformers import Seq2SeqTrainer
import transformers as tf
early_stopping_patience = 10
trainer = Seq2SeqTrainer(
    args=peft_args,
    model=model,
    train_dataset=swahili_final["train"],
    eval_dataset=swahili_final["test"],
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks=[tf.EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)],
)

processor.save_pretrained("./whisper_small/peft/50_dataset_processor")


## 17. Training
print("\n\n Training STARTED..\n\n")

train_result = trainer.train()

## resume from the latest checkpoint
# train_result = trainer.train(resume_from_checkpoint=True)

## resume training from the specific checkpoint in the directory passed
# train_result = trainer.train(resume_from_checkpoint="checkpoint-4000")

print("\n\n Training COMPLETED...\n\n")


## 18. Evaluating & Saving Metrics & Model
print("\n\n Evaluating Model & Saving Metrics...\n\n")

processor.save_pretrained(save_directory="./whisper-small/peft/50_dataset_processor/again")
# trainer.save_model() 

metrics = train_result.metrics
trainer.save_metrics("train", metrics)
trainer.save_state()

metrics = trainer.evaluate(
    metric_key_prefix="eval",
    max_length= peft_args.generation_max_length,
    num_beams= peft_args.generation_num_beams,
)

trainer.save_metrics("eval", metrics)



 Training STARTED..




D:\Downloads\Swahili-translation-English-STT--main\ASR\lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
D:\Downloads\Swahili-translation-English-STT--main\ASR\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
D:\Downloads\Swahili-translation-English-STT--main\ASR\lib\site-packages\torch\utils\checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients 

Step,Training Loss,Validation Loss
400,1.655900,2.087300
800,1.317400,1.905172
1200,1.208500,1.797060
1600,1.303600,1.742837
2000,1.106500,1.715279
2400,0.873500,1.687788
2800,0.740700,1.691673
3200,0.801200,1.681026
3600,0.860900,1.710523
4000,0.504100,1.725510


D:\Downloads\Swahili-translation-English-STT--main\ASR\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
D:\Downloads\Swahili-translation-English-STT--main\ASR\lib\site-packages\torch\utils\checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
D:\Downloads\Swahili-translation-English-STT--main\ASR\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current beha



 Training COMPLETED...




 Evaluating Model & Saving Metrics...




In [ ]:
from peft import PeftModel, PeftConfig
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainer

peft_model_id = "./whisper_small/complete_data_2/checkpoint-6800"
peft_config = PeftConfig.from_pretrained(peft_model_id)
model = WhisperForConditionalGeneration.from_pretrained(
    peft_config.base_model_name_or_path, load_in_8bit=True, device_map="auto"
)
model = PeftModel.from_pretrained(model, peft_model_id)

In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import gc

eval_dataloader = DataLoader(swahili_final["test"], batch_size=8, collate_fn=data_collator)

model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
    with torch.cuda.amp.autocast():
        with torch.no_grad():
            generated_tokens = (
                model.generate(
                    input_features=batch["input_features"].to("cuda"),
                    decoder_input_ids=batch["labels"][:, :4].to("cuda"),
                    max_new_tokens=255,
                )
                .cpu()
                .numpy()
            )
            labels = batch["labels"].cpu().numpy()
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
            metric.add_batch(
                predictions=decoded_preds,
                references=decoded_labels,
            )
    del generated_tokens, labels, batch
    gc.collect()
wer = 100 * metric.compute()
print(f"{wer=}")